In [1]:
import cv2
import time
import numpy as np
from skimage.measure import compare_ssim

# в обох варіантах використаємо підхід, що полягає в видаленні кожного 2 кадру. також всі важливі кадри будемо зберігати в папках decoded, original та differ


In [21]:
cap = cv2.VideoCapture('sample2.avi')

In [3]:
def encode1(cap):
    encoded = []
    ret, previmg = cap.read()
    i = 1
    while (True):
        ret, img = cap.read()
        if (ret == 0): break
        if (i % 2 == 0 ):
            previmg = img
        else:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            prevgray = cv2.cvtColor(previmg, cv2.COLOR_BGR2GRAY)
            flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            code = np.concatenate((previmg, flow), axis = -1)
            encoded.append(code)
        i += 1
        '''
        if (i == 8): 
            encoded = np.array(encoded)
            
            return encoded
        '''
        ch = 0xFF & cv2.waitKey(5)
        if ch == 27:
            break
    return np.array(encoded)

In [4]:
encoded1 = encode1(cap)
with open('encoded4.npy', 'wb') as f:
    np.save(f, encoded1)

# зсуваємо кожен піксель на flow порахований методом farneback

In [5]:
def apply_flow(img, flow):
    tmp = img
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                try:
                    tmp[int(i+flow[i,j,0]),int(j+flow[i,j,1]), k] = img[i,j,k]
                except: pass
                '''
                Exception as e:
                    print(i,j,k)
                    print(img[i,j,k])
                    print(flow[i,j,0], flow[i,j,1])
                    '''
    return tmp

# декодер бере 2 збережених кадра та відновлює, той що був між ними

In [6]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('decoded4.avi',fourcc, 20.0, (480,270))

In [7]:
def decode1(out):
    with open('encoded4.npy', 'rb') as f:
        encoded = np.load(f)

    t1 = time.time()
    i = 1
    for x in encoded:
        img = np.uint8(x[:,:,:3])
        #print(img.shape)
        flow = x[:,:,3:]
        #print(flow.shape)
        pred = apply_flow(img, flow)
        out.write(img)
        out.write(pred)
        i += 2

    t2 = time.time()

    out.release()        
    return (t2 - t1)/i

In [8]:
time1 = decode1(out)

In [9]:
print(time1)

1.1447592940632167


# обраховуємо візуальну різницю між оригіналом та відновленим кадром

# другий метод на основі Lucas-Kanade

In [22]:
def apply_flow2(img, p0, p1):
	tmp = img
	q0 = np.array(p0)
	q1 = np.array(p1)
	e = 2*(q1 - q0)
	for i in range(e.shape[0]):

		try:

			for q in range(int(q0[i,0,0]) - 5, int(q0[i,0,0]) +5):

				for p in range(int(q0[i,0,1]) - 5, int(q0[i,0,1]) +5):

					for k in range(3):

						tmp[q + int(e[i,0,0]), p + int(e[i,0,1]),k] = img[q, p,k]
		except: pass
	#except: print('##')
	return tmp

In [23]:
def encode2(cap, path):
    encoded = []
    
    feature_params = dict( maxCorners = 500,
                                                 qualityLevel = 0.2,
                                                 minDistance = 10,
                                                 blockSize = 20)
    lk_params = dict( winSize  = (15,15),
                                        maxLevel = 2,
                                        criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    color = np.random.randint(0,255,(100,3))
    
    ret, previmg = cap.read()
    
    i = 1
    while (True):
        print(i)
        ret, img = cap.read()
        if (ret == 0): break
        if (i % 2 == 0 ):
            previmg = img
        else:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            prevgray = cv2.cvtColor(previmg, cv2.COLOR_BGR2GRAY)
            p0 = cv2.goodFeaturesToTrack(prevgray, mask = None, **feature_params)
            p1, _, _ = cv2.calcOpticalFlowPyrLK(prevgray, gray, p0, None, **lk_params)
            encoded.append(previmg)
            #print(p0)
            with open(path + f'/p{int(i/2)}.npy', 'wb') as f:
                np.save(f, np.concatenate((p0, p1), axis = 0))
        i += 1
        
        ch = 0xFF & cv2.waitKey(5)
        if ch == 27:
            break
            
    with open(path + '/encoded3.npy', 'wb') as f:
        np.save(f, encoded)
    return

In [24]:
encode2(cap, "encoded_video_3")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [25]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('decoded3.avi',fourcc, 20.0, (480,270))

In [26]:
def decode2(out, path):
    t1 = time.time()
    with open(path+'/encoded3.npy', 'rb') as f:
        encoded = np.load(f)
    encoded = np.uint8(encoded)
        
        
    i = 0
    while(1):
        try:
            with open(path+f'/p{i}.npy', 'rb') as f:
                p = np.load(f)
            p0 = p[:int(p.shape[0]/2)]
            p1 = p[int(p.shape[0]/2):]
            pred = apply_flow2(encoded[i], p0, p1)
            #cv2.imwrite(f'img{i}.jpg', pred)
            out.write(encoded[i])
            #print(encoded[i].shape)
            out.write(pred)
            #print(pred.shape)

            k = cv2.waitKey(30) & 0xff
            if k == 27:
                    break

            i += 1

        except:
            out.release()
            break
        '''
        if (i == 8): 
            out.release() 
            return 
        ''' 
            
            
    t2 = time.time()

    out.release()        
    return (t2 - t1)/i

#  ми бачимо, що farneback працює довше за рахонук зміщення всіх пікселів зображення. але мінусом є те, що не завжди зміщення є правильно нормлваним і пікселі перескакують надто далеко. Lucas-Kanade навпаки не завжди дектектить ключові точки і врезультвті ми бачимо кадр майже ідентичний попередньому. В обох методах є потенціал для покращення, що звичайно ж приведе до більшої складності алгоритму

In [27]:
time2 = decode2(out, "encoded_video_3")

In [28]:
print(time2)

0.002815232276916504


In [29]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('difference.avi',fourcc, 20.0, (480,270))

In [30]:
def difference(cap_d, cap_o, out):
    ret_d, img_d = cap_d.read()
    ret_o, img_o = cap_o.read()
    
    q = 0
    scores = []
    while True:
        
        
            
            
        ret_d, after = cap_d.read()
        ret_o, before = cap_o.read()
        
        if (q%2 == 0): 
            out.write(before)
            q += 1
            continue
        
        if(ret_d == 0 & ret_o == 0): break
        
        #before = cv2.imread(path1 + f"{q}.png", cv2.IMREAD_COLOR)
        #after = cv2.imread(path2 + f"{q}.png", cv2.IMREAD_COLOR)

        # Convert images to grayscale
        before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
        after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

        # Compute SSIM between two images
        (score, diff) = compare_ssim(before_gray, after_gray, full=True)
        #print("Image similarity", score)
        scores.append(score)

        # The diff image contains the actual image differences between the two images
        # and is represented as a floating point data type in the range [0,1] 
        # so we must convert the array to 8-bit unsigned integers in the range
        # [0,255] before we can use it with OpenCV
        diff = (diff * 255).astype("uint8")

    # Threshold the difference image, followed by finding contours to
    # obtain the regions of the two input images that differ
        thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]

        filled_after = after.copy()

        for c in contours:
            area = cv2.contourArea(c)
            if area > 40:
                x,y,w,h = cv2.boundingRect(c)
                cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
                cv2.drawContours(filled_after, [c], 0, (0,150,0), -1)

        for i in range(before.shape[0]):
            for j in range(before.shape[1]):
                for k in range(before.shape[2]):
                    filled_after[i,j,k] = (int(filled_after[i,j,k]) + int(after[i,j,k])) / 2

        out.write(filled_after)
        #cv2.imshow("display", filled_after);
        q += 1
    scores = np.array(scores)
    return np.mean(scores)

In [31]:
cap_d = cv2.VideoCapture('decoded4.avi')
cap_o = cv2.VideoCapture('decoded3.avi')
scores = difference(cap_d, cap_o, out)

<ipython-input-30-fdc7d582b113>:30: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  (score, diff) = compare_ssim(before_gray, after_gray, full=True)


In [32]:
print(scores)

0.9808889643854586
